In [ ]:
MCS

In [ ]:
import numpy as np
import os
import sys
import random
import time
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.special import gamma

# Setup DNN paths and imports
flite_dir = os.path.abspath(os.path.join('..', 'FLITEonPy'))
sys.path.append(flite_dir)

dnn_dir = os.path.abspath(os.path.join('..', 'DNN_CFD'))
sys.path.append(dnn_dir)

from airfoil_generator import generate_airfoil_coordinates
from FLITE2DPY import FLITE2DPY
from DNN_Only import ResidualBlock, ChannelSpecificDecoder, EncoderDecoderCNN, workflow


In [ ]:
# Function to sample a new set of PARSEC parameters within the bounds
def sample_parsec_parameters(baseline, bounds):
    return [np.random.uniform(low, high) for (low, high) in bounds]

# Baseline PARSEC parameters for NACA0012
baseline_parsec = [0.015, 0.3, 0.06, -0.45, 0.3, -0.06, 0.45, 0, 0, 0.02, 19.5]
# ±20% bounds for each parameter
bounds = [(value * 0.8, value * 1.2) for value in baseline_parsec]


In [ ]:
start_total_p = time.time()

# Constants
num_nests = 30  # Number of nests
A = 1  # Max Lévy flight step size
phi = (1 + np.sqrt(5)) / 2  # Golden Ratio
max_evaluations = 20  # Maximum number of objective evaluations
n_abandon = int(0.75 * num_nests)  # Number of nests to be abandoned
top_nests_frac = 0.25  # Fraction of nests considered top nests

#Flow Parameters
mach = 0.5
aoa = 4

# Paths for storing generated data
sdf_path = './SDF'
geo_path = './Airfoils'
dnn_output_path = './Inference'

# Function to evaluate the fitness of a nest (airfoil)
def fitness_function(airfoil_number, mach, aoa):
    C_L, C_D = FLITE2DPY(airfoil_number, mach, aoa, geo_path)
    if np.isnan(C_L) or np.isnan(C_D):
        return 0  # If any component or ratio is NaN, return a fitness of 0
    return C_L / C_D

# Function to evaluate the fitness of a nest (airfoil)
def fitness_function_dnn(airfoil_number, mach, aoa):
    C_L, C_D = workflow(airfoil_number, mach, aoa, sdf_path, geo_path, dnn_output_path)
    if (C_L / C_D) > 100:
        return None  
    return C_L / C_D

# Initial population setup
population = []
fitness_values = []

for i in range(num_nests):
    while True:
        # Sample a new set of PARSEC parameters
        sampled_parsec = sample_parsec_parameters(baseline_parsec, bounds)
        generate_airfoil_coordinates(sampled_parsec, i + 1)  # Save airfoil as i+1.txt

        # Compute fitness
        fitness = fitness_function_dnn(i + 1, mach, aoa)# Using DNN for lift coefficient as fitness
        if fitness is not None:
                break
                
    population.append(sampled_parsec)
    fitness_values.append(fitness)

# Track the number of evaluations
num_evaluations = num_nests

# Initial generation count
generation = 0

# Levy flight function for generating new solutions
def levy_flight(beta=1.5):
    sigma_u = (gamma(1 + beta) * np.sin(np.pi * beta / 2) / (gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
    u = np.random.normal(0, sigma_u, size=len(bounds))
    v = np.random.normal(0, 1, size=len(bounds))
    step = u / (np.abs(v) ** (1 / beta))
    return step

# Total execution time
end_total_p = time.time()
elapsed_total_p = end_total_p - start_total_p

print(fitness_values)
print(f'Total time taken to initialise population: {elapsed_total_p:.2f} seconds')


In [ ]:
start_total = time.time()
# Main MCS Loop

# Initialize lists for plotting
generations = []
best_fitness_values = []
best_population_values = []
population_archive = []
fitness_archive = []

while generation < max_evaluations:
    generation += 1
    
    if generation >= 16:
        # Sort nests by fitness (descending order)
        paired_population = list(zip(population, fitness_values))
        paired_population.sort(key=lambda x: x[1], reverse=True)
        population, fitness_values = zip(*paired_population)

        # Convert tuples back to lists
        population = list(population)
        fitness_values = list(fitness_values)

        generations.append(generation)
        best_fitness_values.append(fitness_values[0])
        best_population_values.append(population[0])
        population_archive.append(population)
        fitness_archive.append(fitness_values)
        print(fitness_values)
        
        # Abandonment phase: Replace some nests with new solutions
        for i in range(n_abandon):
            xi = population[-(i + 1)]  # Select the nest to be abandoned
            alpha = A / np.sqrt(generation)  # Lévy flight step size
            step = levy_flight()
            xk = xi + alpha * step  # New egg via Lévy flight
            xk = np.clip(xk, [low for (low, high) in bounds], [high for (low, high) in bounds])
            generate_airfoil_coordinates(xk, i + 1 + num_nests)  # Save as new airfoil
            fitness = fitness_function(i + 1 + num_nests, mach, aoa)

            population[-(i + 1)] = xk
            fitness_values[-(i + 1)] = fitness
            num_evaluations += 1

        # Refinement of top nests
        #num_top_nests = int(top_nests_frac * num_nests)
        num_top_nests = num_nests - n_abandon
        for i in range(num_top_nests):
            xi = population[i]
            xj_index = np.random.choice(num_top_nests)
            xj = population[xj_index]

            if np.array_equal(xi, xj):
                alpha = A / (generation**2)  # Lévy flight step size for same nest
                step = levy_flight()
                xk = xi + alpha * step
                xk = np.clip(xk, [low for (low, high) in bounds], [high for (low, high) in bounds])
            else:
                # Calculate dx
                dx = np.abs(np.array(xi) - np.array(xj)) / phi

                # Determine the direction of movement based on fitness
                fitness_xi = fitness_values[i]
                fitness_xj = fitness_values[xj_index]

                if fitness_xi < fitness_xj:
                    # xi is worse, move towards xj
                    xk = np.array(xi) + dx * np.sign(np.array(xj) - np.array(xi))
                else:
                    # xj is worse, move towards xi
                    xk = np.array(xj) + dx * np.sign(np.array(xi) - np.array(xj))

                # Clip new solution within bounds
                xk = np.clip(xk, [low for (low, high) in bounds], [high for (low, high) in bounds])

            # Generate airfoil for new solution xk and evaluate its fitness
            generate_airfoil_coordinates(xk, num_nests + num_evaluations + 1)  # Save as new airfoil
            fitness = fitness_function(num_nests + num_evaluations + 1, mach, aoa)

            # Compare and replace if new solution is better
            l = np.random.choice(num_nests)
            if fitness > fitness_values[l]:
                population[l] = xk
                fitness_values[l] = fitness
                num_evaluations += 1

        # Output the best nest and its fitness value
        best_nest = population[0]
        best_fitness = fitness_values[0]
        print(f"Generation: {generation}, Best Fitness: {best_fitness}")
    else:
        if generation % 5 == 0:
            # Sort nests by fitness (descending order)
            paired_population = list(zip(population, fitness_values))
            paired_population.sort(key=lambda x: x[1], reverse=True)
            population, fitness_values = zip(*paired_population)

            # Convert tuples back to lists
            population = list(population)
            fitness_values = list(fitness_values)

            generations.append(generation)
            best_fitness_values.append(fitness_values[0])
            best_population_values.append(population[0])
            population_archive.append(population)
            fitness_archive.append(fitness_values)
            print(fitness_values)
            
            # Abandonment phase: Replace some nests with new solutions
            for i in range(n_abandon):
                xi = population[-(i + 1)]  # Select the nest to be abandoned
                alpha = A / np.sqrt(generation)  # Lévy flight step size
                step = levy_flight()
                xk = xi + alpha * step  # New egg via Lévy flight
                xk = np.clip(xk, [low for (low, high) in bounds], [high for (low, high) in bounds])
                generate_airfoil_coordinates(xk, i + 1 + num_nests)  # Save as new airfoil
                fitness = fitness_function(i + 1 + num_nests, mach, aoa)

                population[-(i + 1)] = xk
                fitness_values[-(i + 1)] = fitness
                num_evaluations += 1

            # Refinement of top nests
            num_top_nests = num_nests - n_abandon
            for i in range(num_top_nests):
                xi = population[i]

                generate_airfoil_coordinates(xi, i + 1 + num_nests)  # Save as new airfoil
                fitness = fitness_function(i + 1 + num_nests, mach, aoa)

                population[i] = xi
                fitness_values[i] = fitness
                num_evaluations += 1

            for i in range(num_top_nests):
                xi = population[i]
                xj_index = np.random.choice(num_top_nests)
                xj = population[xj_index]

                if np.array_equal(xi, xj):
                    alpha = A / (generation**2)  # Lévy flight step size for same nest
                    step = levy_flight()
                    xk = xi + alpha * step
                    xk = np.clip(xk, [low for (low, high) in bounds], [high for (low, high) in bounds])
                else:
                    # Calculate dx
                    dx = np.abs(np.array(xi) - np.array(xj)) / phi

                    # Determine the direction of movement based on fitness
                    fitness_xi = fitness_values[i]
                    fitness_xj = fitness_values[xj_index]

                    if fitness_xi < fitness_xj:
                        # xi is worse, move towards xj
                        xk = np.array(xi) + dx * np.sign(np.array(xj) - np.array(xi))
                    else:
                        # xj is worse, move towards xi
                        xk = np.array(xj) + dx * np.sign(np.array(xi) - np.array(xj))

                    # Clip new solution within bounds
                    xk = np.clip(xk, [low for (low, high) in bounds], [high for (low, high) in bounds])

                # Generate airfoil for new solution xk and evaluate its fitness
                generate_airfoil_coordinates(xk, num_nests + num_evaluations + 1)  # Save as new airfoil
                fitness = fitness_function(num_nests + num_evaluations + 1, mach, aoa)

                # Compare and replace if new solution is better
                l = np.random.choice(num_nests)
                if fitness > fitness_values[l]:
                    population[l] = xk
                    fitness_values[l] = fitness
                    num_evaluations += 1

            # Output the best nest and its fitness value
            best_nest = population[0]
            best_fitness = fitness_values[0]
            print(f"Generation: {generation}, Best Fitness: {best_fitness}")
        else:
            # Sort nests by fitness (descending order)
            paired_population = list(zip(population, fitness_values))
            paired_population.sort(key=lambda x: x[1], reverse=True)
            population, fitness_values = zip(*paired_population)

            # Convert tuples back to lists
            population = list(population)
            fitness_values = list(fitness_values)

            generations.append(generation)
            best_fitness_values.append(fitness_values[0])
            best_population_values.append(population[0])
            population_archive.append(population)
            fitness_archive.append(fitness_values)
            print(fitness_values)
            
            # Abandonment phase: Replace some nests with new solutions
            for i in range(n_abandon):
                while True:
                    xi = population[-(i + 1)]  # Select the nest to be abandoned
                    alpha = A / np.sqrt(generation)  # Lévy flight step size
                    step = levy_flight()
                    xk = xi + alpha * step  # New egg via Lévy flight
                    xk = np.clip(xk, [low for (low, high) in bounds], [high for (low, high) in bounds])
                    generate_airfoil_coordinates(xk, i + 1 + num_nests)  # Save as new airfoil
                    fitness = fitness_function_dnn(i + 1 + num_nests, mach, aoa)
                    if fitness is not None:
                        break

                population[-(i + 1)] = xk
                fitness_values[-(i + 1)] = fitness
                num_evaluations += 1

            # Refinement of top nests
            num_top_nests = num_nests - n_abandon
            for i in range(num_top_nests):
                xi = population[i]
                xj_index = np.random.choice(num_top_nests)
                xj = population[xj_index]

                while True:
                    alpha = A / (generation**2)  # Lévy flight step size for same nest
                    step = levy_flight()
                    xk = xi + alpha * step
                    xk = np.clip(xk, [low for (low, high) in bounds], [high for (low, high) in bounds])

                    # Generate airfoil for new solution xk and evaluate its fitness
                    generate_airfoil_coordinates(xk, num_nests + num_evaluations + 1)  # Save as new airfoil
                    fitness = fitness_function_dnn(num_nests + num_evaluations + 1, mach, aoa)

                    if fitness is not None:
                        break

                # Compare and replace if new solution is better
                l = np.random.choice(num_nests)
                if fitness > fitness_values[l]:
                    population[l] = xk
                    fitness_values[l] = fitness
                    num_evaluations += 1

            # Output the best nest and its fitness value
            best_nest = population[0]
            best_fitness = fitness_values[0]
            print(f"Generation: {generation}, Best Fitness: {best_fitness}")
        
# Total execution time
end_total = time.time()
elapsed_total = end_total - start_total


In [ ]:
print(f'Total time taken to initialise population: {elapsed_total_p:.2f} seconds')
print(f'Total time taken for optimisation: {elapsed_total:.2f} seconds')

# Final best solution
print(f"Final Best Fitness: {best_fitness}")
print(f"Best PARSEC Parameters: {best_nest}")


In [ ]:
# Plot fitness progression
plt.figure(figsize=(10, 6))
plt.plot(generations, best_fitness_values, marker='o')
plt.xlabel('Generation')
plt.ylabel('Best Fitness Value')
plt.title('Fitness Progression Over Generations')
plt.grid(True)
plt.show()


In [ ]:
# Assume parameters are in a list of lists
# Each element in population is a list of 11 parameters
num_params = len(best_population_values[0])
for i in range(num_params):
    param_values = [p[i] for p in best_population_values]
    plt.figure(figsize=(10, 6))
    plt.plot(generations, param_values, marker='o')
    plt.xlabel('Generation')
    plt.ylabel(f'Parameter {i+1} Value')
    plt.title(f'Evolution of Parameter {i+1} Over Generations')
    plt.grid(True)
    plt.show()


In [ ]:
# Convert population to a DataFrame for easy plotting
df = pd.DataFrame(best_population_values, columns=[f'Param {i+1}' for i in range(num_params)])

# Add fitness values to the DataFrame
df['Fitness'] = best_fitness_values

# Remove duplicates based on parameter columns
df_1 = df.drop_duplicates(subset=[f'Param {i+1}' for i in range(num_params)])

# Pairwise plots for the first few parameters, colored by fitness
sns.pairplot(df_1, hue='Fitness', palette='turbo')
plt.suptitle('Pairwise Parameter Relationships', y=1.02)
plt.show()


In [ ]:
# Print the DataFrame after removing duplicates
print(df_1.head())
print(df_1.shape)

# Proceed with pairplot as before
sns.pairplot(df_1, hue='Fitness', palette='turbo', diag_kind="hist", corner=True)
plt.suptitle('Pairwise Parameter Relationships', y=1.02)
plt.show()


In [ ]:
# Convert fitness_values to a NumPy array for easier handling
fitness_values_array = np.array(best_fitness_values)

for i in range(num_params):
    plt.figure(figsize=(10, 6))
    plt.scatter(df[f'Param {i+1}'], fitness_values_array, c=fitness_values_array, cmap='turbo', edgecolor='k')
    plt.xlabel(f'Parameter {i+1}')
    plt.ylabel('Fitness Value')
    plt.title(f'Fitness vs Parameter {i+1}')
    plt.colorbar(label='Fitness Value')
    plt.grid(True)
    plt.show()


In [ ]:
for i in range(num_params):
    plt.figure(figsize=(10, 6))
    plt.hist(df.iloc[:, i], bins=20, edgecolor='k')
    plt.xlabel(f'Parameter {i+1} Value')
    plt.ylabel('Frequency')
    plt.title(f'Distribution of Parameter {i+1}')
    plt.grid(True)
    plt.show()


In [ ]:
# Correlation Analysis
# Convert archives to numpy arrays
poparch = np.array(population_archive)  # Shape: (generations, nests, params)
fitarch = np.array(fitness_archive)      # Shape: (generations, nests)

# Flatten the arrays for correlation analysis
params_flat = poparch.reshape(-1, poparch.shape[-1])
fitness_flat = fitarch.flatten()

# Create DataFrame for correlation calculation
data = pd.DataFrame(params_flat, columns=[f'Param_{i+1}' for i in range(poparch.shape[-1])])
data['Fitness'] = fitness_flat

# Compute correlations
correlations = data.corr()['Fitness'].drop('Fitness')

# Plot the correlations
plt.figure(figsize=(10, 6))
correlations.plot(kind='bar')
plt.xlabel('Parameter')
plt.ylabel('Correlation with Fitness')
plt.title('Parameter Influence on Fitness')
plt.show()


In [ ]:
#Feature Importance Using Machine Learning
from sklearn.ensemble import RandomForestRegressor

# Flatten the population and fitness data
X = poparch.reshape(-1, poparch.shape[-1])
y = fitarch.flatten()

# Train a random forest regressor
model = RandomForestRegressor()
model.fit(X, y)

# Extract feature importances
importances = model.feature_importances_

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.bar(range(len(importances)), importances)
plt.xlabel('Parameter Index')
plt.ylabel('Importance')
plt.title('Feature Importance from Random Forest')
plt.show()


In [ ]:
#Fitness Evolution Over Generations
# Extracting the best, worst, and average fitness per generation
best_fitness = np.max(fitarch, axis=1)
average_fitness = np.mean(fitarch, axis=1)
worst_fitness = np.min(fitarch, axis=1)

plt.figure(figsize=(10, 5))
plt.plot(best_fitness, label='Best Fitness', marker='o')
plt.plot(average_fitness, label='Average Fitness', marker='x')
plt.plot(worst_fitness, label='Worst Fitness', marker='s')
plt.xlabel('Generation')
plt.ylabel('Fitness Value')
plt.title('Fitness Evolution Over Generations')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
#Distribution of Fitness Values

# Convert fitarch to a list of lists
fitarch_list = fitarch.tolist()

plt.figure(figsize=(10, 5))
plt.boxplot(fitarch_list, labels=[f'{i+1}' for i in range(len(fitarch_list))])
plt.xlabel('Generation')
plt.ylabel('Fitness Value')
plt.title('Fitness Distribution Across Generations')
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 5))
plt.boxplot(fitarch_list, labels=[f'{i+1}' for i in range(len(fitarch_list))])
plt.xlabel('Generation')
plt.ylabel('Fitness Value')
plt.ylim(-10,120)
plt.title('Fitness Distribution Across Generations')
plt.grid(True)
plt.show()

In [ ]:
#Evolution of Individual Parameters
num_params = poparch.shape[2]
for param_idx in range(num_params):
    plt.figure(figsize=(10, 5))
    param_values = poparch[:, :, param_idx]
    mean_values = np.mean(param_values, axis=1)
    std_values = np.std(param_values, axis=1)
    
    plt.plot(mean_values, label=f'Parameter {param_idx + 1} Mean', marker='o')
    plt.fill_between(range(len(mean_values)), mean_values - std_values, mean_values + std_values, alpha=0.2)
    plt.xlabel('Generation')
    plt.ylabel('Parameter Value')
    plt.title(f'Evolution of Parameter {param_idx + 1}')
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
from scipy.spatial.distance import pdist, squareform
#Diversity of the Population
for gen_idx in range(poparch.shape[0]):
    generation_data = poparch[gen_idx]
    distance_matrix = squareform(pdist(generation_data))
    
    plt.figure(figsize=(8, 6))
    plt.imshow(distance_matrix, cmap='hot', interpolation='nearest')
    plt.colorbar()
    plt.title(f'Generation {gen_idx + 1} Diversity (Distance Matrix)')
    plt.xlabel('Nest Index')
    plt.ylabel('Nest Index')
    plt.grid(False)
    plt.show()


In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# Sample data preparation
generations = np.array(generations)  # Shape: (N,)
num_nests = len(fitness_archive[0])  # Number of nests
num_generations = len(fitness_archive)  # Number of generations

# Flattening the fitness archive to fit the grid
fitness_matrix = np.array(fitness_archive)  # Shape: (7, 5)
X, Y = np.meshgrid(range(num_nests), range(num_generations))

# Flattening for contour plot
X_flat = X.flatten()
Y_flat = Y.flatten()
Z_flat = fitness_matrix.flatten()

# 3D Plot
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Plotting the surface
sc = ax.scatter(X_flat, Y_flat, Z_flat, c=Z_flat, cmap='turbo', marker='o')
plt.colorbar(sc, ax=ax, label='Fitness Value')
sc.set_clim(0,300)
ax.set_zlim(0,300)
ax.set_xlabel('Nest Index')
ax.set_ylabel('Generation')
ax.set_zlabel('Fitness Value')
ax.set_title('Fitness Value of Nests Across Generations')

plt.show()

# 3D Plot
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(X_flat, Y_flat, Z_flat, c=Z_flat, cmap='turbo', marker='o')
plt.colorbar(sc, ax=ax, label='Fitness Value')
sc.set_clim(0,300)
ax.set_zlim(0,120)
ax.set_xlabel('Nest Index')
ax.set_ylabel('Generation')
ax.set_zlabel('Fitness Value')
ax.set_title('Fitness Value of Nests Across Generations')

plt.show()


In [ ]:
#contour plot

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Create a 3D contour plot
contour = ax.contourf3D(X, Y, fitness_matrix, 1000, cmap='turbo', levels=np.linspace(0, 350, 3000))
plt.colorbar(contour, ax=ax, label='Fitness Value')
ax.set_xlabel('Nest Index')
ax.set_ylabel('Generation')
ax.set_zlabel('Fitness Value')
ax.set_title('Fitness Contour of Nests Across Generations')

plt.show()


In [ ]:
# Paths for SDF, Airfoils, and DNN Output
sdf_path = './SDF'
geo_path = './Airfoils'
dnn_output_path = './Inference'

#Flow Parameters
mach = 0.5
aoa = 4
    
sampled_parsec =  [ 1.20000000e-02,  3.24128270e-01,  5.71671843e-02,-5.40000000e-01,
  3.51931012e-01, -7.20000000e-02 , 3.60000000e-01 , 0.00000000e+00,
  0.00000000e+00,  1.90557281e-02 , 1.85370405e+01]
generate_airfoil_coordinates(sampled_parsec, 10000)
C_L, C_D = workflow(10000, mach, aoa, sdf_path, geo_path, dnn_output_path)
print(f"Coefficients (C_L), (C_D), (C_L/C_D) for airfoil {1}: {C_L}, {C_D}, {C_L/C_D}")


In [ ]:
# Paths for SDF, Airfoils, and DNN Output
geo_path = './Airfoils'

#Flow Parameters
mach = 0.5
aoa = 4
    
sampled_parsec = [ 1.20000000e-02,  3.24128270e-01,  5.71671843e-02,-5.40000000e-01,
  3.51931012e-01, -7.20000000e-02 , 3.60000000e-01 , 0.00000000e+00,
  0.00000000e+00,  1.90557281e-02 , 1.85370405e+01]
generate_airfoil_coordinates(sampled_parsec, 40000)
C_L, C_D = FLITE2DPY(40000, mach, aoa, geo_path)
print(f"Coefficients (C_L), (C_D), (C_L/C_D) for airfoil: {C_L}, {C_D}, {C_L/C_D}")


In [ ]:
import pickle

# Save lists to a file
with open('ddf.pkl', 'wb') as file:
    pickle.dump({
        'generations': generations,
        'best_fitness_values': best_fitness_values,
        'best_population_values': best_population_values,
        'population_archive': population_archive,
        'fitness_archive': fitness_archive
    }, file)